# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
#ADDED CELL
establishments.find_one()

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query1 = {'scores.Hygiene': 20}
fields1 = {'BusinessName': 1, 'scores.Hygiene': 1, '_id': 0}

results1 = list(establishments.find(query1, fields1))

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query1))

# Display the first document in the results using pprint
pprint(results1[0])

In [ ]:
# Convert the result to a Pandas DataFrame
results1_df = pd.json_normalize(results1)
# Display the number of rows in the DataFrame
pprint(results1_df.shape[0])
# Display the first 10 rows of the DataFrame
results1_df.head(10)

In [ ]:
#ADDED CELL
#1. Which establishments have a hygiene score equal to 20?
# Answer: see the full list below
results1_df

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte':4}}
fields2 = {'LocalAuthorityName':1,
         'RatingValue':1, '_id': 0}

results2 = list(establishments.find(query2, fields2))
# Use count_documents to display the number of documents in the result
pprint(establishments.count_documents(query2))
# Display the first document in the results using pprint
pprint(results2[0])

In [ ]:
# Convert the result to a Pandas DataFrame
results2_df = pd.json_normalize(results2)
# Display the number of rows in the DataFrame
pprint(results2_df.shape[0])
# Display the first 10 rows of the DataFrame
results2_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Get Penang Flavours document
query_penang_flavours = {'BusinessName': "Penang Flavours"}
penang_result = establishments.find_one(query_penang_flavours)

degree_search = 0.01
latitude = penang_result['geocode']['latitude']
longitude = penang_result['geocode']['longitude']

query3 = {'RatingValue': 5,
            'geocode.latitude': {
                '$gte': latitude - degree_search, 
                '$lte': latitude + degree_search
                },
            'geocode.longitude': {
                '$gte': longitude - degree_search, 
                '$lte': longitude + degree_search
             }
        }
fields3 = {'RatingValue': 1, 
          'geocode.latitude': 1, 
          'geocode.longitude': 1, 
          'scores.Hygiene': 1,
          'BusinessName': 1,
          '_id':0}

sort3 =  [('scores.Hygiene', 1)]

results3 = list(establishments.find(query3, fields3).sort(sort3).limit(5))
# Print the results
pprint(results3)

In [ ]:
# Convert result to Pandas DataFrame
top5_london_df = pd.json_normalize(results3)
pprint(top5_london_df.shape[0])
top5_london_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query4 = {'$match': {
                    'scores.Hygiene': 0}
              }

# Print the number of documents in the result
group_query4 = {'$group': {
    '_id':{
        'LocalAuthorityName': '$LocalAuthorityName'
    },
    'count':{'$sum': 1}
  }
}
sort_values4 = {'$sort': {'count': -1}}
pipeline = [match_query4, group_query4, sort_values4]
results4 = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results4[:10])

In [ ]:
# Convert the result to a Pandas DataFrame
results4_df = pd.json_normalize(results4)
results4_df = results4_df.rename(columns={'_id.LocalAuthorityName':'_id'})
# Display the number of rows in the DataFrame
pprint(results4_df.shape[0])
# Display the first 10 rows of the DataFrame
results4_df.head(10)